#PLEASE SEE BELOW FOR TASK QUESTIONS

#Remarks

Some entries in SEHeadline are generically called "Q4 2020 earnings call", so not directly containing any company's name; a potential solution may be to include the name from SECompanyName, even though this may lead to inconsistencies, since it only hosts the most recent ones, and such headline could refer to a moment in time when the firm was called differently.

For this reason, CRSP's date column could be checked to make sure that at the call's time, the company's name was indeed the one suggested by SECompanyName, otherwise it should be fixed.
<br><br><br>

Using extractOne() initially returned the warning:

"*Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning.*"

I solved it by installing python-Levenshtein, which also decreased processing time of ~200:30 rows (se:crsp) from 6 seconds to .6 seconds (only works locally, still slow on colab).
<br><br><br>

Couldn't make pandarallel work from colab, eventually managed to get it to work locally using spyder on WSL, since I'm on Windows.
<br><br><br>

Even though the code below directly refers to se_us for extractOne(), I actually split the dataframe into several 50k row tranches, else parallel_apply() would have run for around 3 to 4 days to process the whole dataframe at once, and I couldn't afford to keep the computer turned on for such a long time.

I then concatenated the 50k rows tranches and formed se_us again.
<br><br><br>

A final note on execution time, when using parallel_apply() for extractOne(), I processed 10 rows from SE with all rows from CRSP in 9 seconds, yielding a speed of approximately 1.11 rows/sec; keeping the same size for CRSP, other references I obtained using %%time were:

20 rows in 15 seconds, returning 1.33 rows/sec;

100 rows in ~96 seconds, returning 1.04 rows/sec;

1000 rows in ~780 seconds, returning 1.28 rows/sec;

10000 rows in ~9400 seconds, returning 1.06 rows/sec;

40000 rows in ~31020 seconds, returning 1.29 rows/sec;

50000 rows in ~38160 seconds, returning 1.31 rows/Sec

Having maintained the computer approximately in the same conditions at each execution, process-wise, could it be that, depending on the rows selected, some strings are longer than others and thus require more time for processing, yielding a lower speed? Otherwise I can't explain why some executions returned approximately 1.04 rows/sec, while others 1.33 rows/sec.

#Description

Initially I import the needed packages and load the data, then do some initial checks to make sure the data is prepared to be processed; I drop some SEid duplicates which were included into the main dataset, because even though they don't impact on the final result, since if the algorithm correctly matches an entry, it will also match its duplicate, it still marginally reduces the dataset's size.

In [ ]:
!pip install thefuzz

In [ ]:
!pip install pandarallel

In [ ]:
!pip install python-Levenshtein

In [ ]:
import numpy as np
import pandas as pd
import re
import time
import nbconvert

from thefuzz import fuzz
from thefuzz import process

from pandarallel import pandarallel
pandarallel.initialize(nb_workers = 8, progress_bar = False)

In [ ]:
crsp = pd.read_csv(r"CRSPnames.csv")
se = pd.read_csv(r"SEmappingsDAFA.csv")

In [ ]:
print(crsp.isnull().sum())

print(se.isnull().sum())

print(se["SEid"].duplicated().sum())

print(se["SEHeadline"].duplicated().sum())

Conceptually, CRSP could have duplicates in any column, but SE shouldn't have duplicates in SEId and in SEHeadline, since theoretically each row should identify a single company's specific earnings call.

In practice, both SEId and SEHeadline columns contain duplicates; the former can be safely removed as they identify the same earnings call for the same company, whereas the latter shouldn't be dropped, as they identify an earnings call from the same quarter and year, but related to two different companies.

Omitting these would cause information loss, and thus such action must be avoided; a potential solution has been highlighted in the Remarks section, namely appending the company's name to such "generic" headlines, so that they would cease being duplicates, although this would potentially cause mismatches were the company name not coherent in time.

Given that there are 2213 SEId duplicates and 2891 SEHeadline duplicates, their difference could represent, as written above, the amount of lines related to earnings calls for different companies, but which have a generic name and for which appending the company's name could be useful.

In [ ]:
se = se.drop_duplicates(subset = ["SEid"], keep = "first", ignore_index = True)

It's interesting to notice that apparently there's one more duplicate value compared to the difference 2891 - 2213 = 678, resulting from headline - id duplicates, why is this the case?

Having manually investigated the dataframe with 679 duplicated entries, I have noticed rows containing information about the same earnings call, for the same company, but tied to different SEids (ex. id 840225 and 957435, both related to Q3 2004 Dr. Reddy's Laboratories Earnings Conference Call, ticker DRREDDY.NSE and company name Dr.Reddy's Laboratories Ltd).

This seems to be in contrast with the statement about the id column containing unique earnings calls identifiers, but may not be relevant for merging the dataframes.

In [ ]:
print(se["SEHeadline"].duplicated().sum())

I then split SE dataset into 2 parts: US and non US sections, according to the most frequent american business entities abbreviations, such as INC or CO, grouped with some more "typically american" ones such as PLC or TRUST, even though they appear less often within CRSP, as the code below shows, where the assumption is that the last word within COMNAM indeed represents the entity, which is not necessarily the case for every entry.

In [ ]:
crsp["ENTITY"] = crsp["COMNAM"].apply(lambda x: re.findall(r"\w+", x)[-1])

entity_lst = crsp["ENTITY"].value_counts()


I also had an idea to filter out companies whose ticker isn't linked to american exchanges, but I couldn't come up with an extensive list of the potential exchanges, and I thought that anyways some american companies may also be negotiated outside american ones; another problem is that many entries do not contain an indication of the exchange where the company is negotiated (ex. 1-800 Flowers.com has some entries where its ticker is FLWS, and others where its ticker is FLWS.OQ), hence I preferred moving forward with the entity approach.

Even though originally I subsequently separated se_us according to the presence of special characters (whitespace excluded) in the SEHeadline column,
because I obtained better results in the test dataframes by using the partial ratio scorer instead of the default WRatio one, I eventually decided
not to do it anymore and just kept a single US section, to then remove any such special character and run extractOne() with the default scorer, which yielded a better overall accuracy in the test dataframes I checked manually; those are attached to the ZIP, if you want to double check.

In [ ]:
se_us = se[se["SECompanyName"].str.contains("|".join([" INC.", " INC", " Inc.", " Inc",
                                                   " LTD.", " LTD", " Ltd", " LP",
                                                   " PLC", " Plc", ".COM", ".Com"
                                                   " CO.", " CO", " Co.", " Co",
                                                   " CORP", " CORP.", " Corp", " Corp.",
                                                   " LLC", " Llc", " Lp", " Group",
                                                   " Trust", " plc"]))]
se_us = se_us.reset_index(drop = True)

se_non_us = se[~se["SECompanyName"].str.contains("|".join([" INC.", " INC", " Inc.", " Inc",
                                                   " LTD.", " LTD", " Ltd", " LP",
                                                   " PLC", " Plc", ".COM", ".Com"
                                                   " CO.", " CO", " Co.", " Co",
                                                   " CORP", " CORP.", " Corp", " Corp.",
                                                   " LLC", " Llc", " Lp", " Group",
                                                   " Trust", " plc"]))]
se_non_us = se_non_us.reset_index(drop = True)

se_non_us["MergeComnam"] = "NON US COMPANY"

This initial filtering is based on the content of "SECompanyName", and not on "SEHeadline", because often the latter column may contain the company's name
without its entity, leading to wrongly omitting american companies from the dataframe; this, for instance, is the case for Apple Inc, which appears as
Apple Computer in many entries in SEHeadline prior to 2007, and not as Apple Computer Inc; in fact, having tried to filter on this column, I initially
obtained 140k entries, versus the 275k I eventually kept when filtering on "SECompanyName".

The datasets have their indexes reset, also further down the code, because this sensibly decreases the execution time and overall provides tidier data,
which is easier to slice if needed.

I then proceed to define one helper function to extract random datasets of length 1000, which will serve to test the algorithm and the preprocessing
that I carry out on SEHeadline.

Overall I repeated this step 5 times, and in the previous code I wrote for the task I tested 6 other dataframes of such
size, totalling 11000 rows out of approximately 275000 from the US section, namely 4% of the US dataframe; I'm not sure this is enough to infer a result
for the matching accuracy, but I assume it is.


In [ ]:
def rand_df(df_start):
    start = np.random.randint(0, len(df_start) - 1000)
    end = start + 1000
    df_rand = df_start.iloc[start:end].copy()
    
    return df_rand

Another function aims at cleaning SEHeadline, with the goal of making each entry as close as possible to CRSP's counterpart,
for example by removing the quarter and year information, and any special character which is not usually included into CRSP.

Other "distracting" words such as "Interim" or "Half Year" are also removed at this stage, although the impact in terms of accuracy wasn't large in my test dataframes, but I decided to omit them; other potentially distracting words such as "And" or "Discuss", which appear in the original dataframe weren't removed, because the resulting list of all potentially distracting words could be very large for manual deletion.

An algorithm could be conceived to recognize such words as potentially distracting, and remove them based on their frequency, or simply remove all of them, although this could lead to wrongly removing words which
constitute the company's name.


In [ ]:
def clean_df(df):
    
    df["SEHeadline_fix"] = df["SEHeadline"].str.split(" Earnings", 1).str[0].str.strip()
    df["SEHeadline_fix"] = df["SEHeadline_fix"].str.replace(r"\bQ[1-4]\b", " ")
    df["SEHeadline_fix"] = df["SEHeadline_fix"].str.replace(r"\b2[0-9]{3}\b", " ")
    df["SEHeadline_fix"] = df["SEHeadline_fix"].str.replace(r"[^a-zA-Z\d\s:]", " ")
    
    df["SEHeadline_fix"] = df["SEHeadline_fix"].str.replace("|".join([
                              "Half Year", "Full Year", "Interim", "Preliminary", "Interim",
                              "Conference", "First Quarter", "Second Quarter", "Third Quarter",
                              "Fourth Quarter", "Results ", "Call", "Fiscal", "FY"]), " ")
    
    df["SEHeadline_fix"] = df["SEHeadline_fix"].str.replace("Corporation", "Corp")
    df["SEHeadline_fix"] = df["SEHeadline_fix"].str.replace("Limited", "Ltd")
    
    return df

The last helper function applies the aforementioned ones to the starting dataframe, which in this case is se_us, to prepare the test dataframe for
extractOne.


In [ ]:
def prep_df(df_start):
    final_df = rand_df(df_start)
    final_df = clean_df(final_df)
    final_df = final_df.reset_index(drop = True)
    
    return final_df

I call the final helper function to create a test dataframe to showcase the results, and run extractOne() to get the best match according to WRatio, the default scorer.

The output is then written in csv after dropping the helper column "SEHeadline_fix", used to merge with CRSP, with all the processing done referring to the entire CRSP dataset, meaning no slices were considered to infer results, which could otherwise be biased.

The test dataframes highlight an average matching accuracy of 71% (given respectively as an average of 70% from the 6 test dataframes I manually sliced,
plus an average of 72% stemming from the 5 random ones I selected, composed of 49%, 83%, 57%, 91% and 84%); I assumed this as an accuracy measure to
estimate that around 70% of the companies will be correctly matched.

In [ ]:
df_test = prep_df(se_us)

In [ ]:
%%time
df_test["MergeComnam"] = df_test["SEHeadline_fix"].parallel_apply(lambda x: process.extractOne(x, list(crsp["COMNAM"]))[0])

In [ ]:
df_test = df_test.drop(columns = "SEHeadline_fix")

df_test.to_csv("/home/edoardo/df_test.csv", index = False)


I eventually apply the clean_df function to se_us, the total filtered dataframe containing data about US companies, to prepare it for extractOne(), after having estimated the matching accuracy in the previous steps, and then run the matching function.

At this stage, I also drop the helper column from se_us, and check for missing values to see whether some entries were left out; at this point I realized this is not the case, even though I thought I would have some because of a warning message I received while running parallel_apply(), namely:

"*Applied processor reduces input query to empty string, all comparisons will
have score 0.*"

In [ ]:
se_us = clean_df(se_us)

In [ ]:
%%time
se_us["MergeComnam"] = se_us["SEHeadline_fix"].parallel_apply(lambda x: process.extractOne(x, list(crsp["COMNAM"]))[0])

In [ ]:
se_us = se_us.drop(columns = "SEHeadline_fix")

se_us["MergeComnam"].isnull().sum()

I merge back together se_us with se_non_us to get the original dataframe, filtered for duplicates, where now american companies are matched with their counterparts on CRSP, and non american companies are highlighted in the MergeComnam column as "NON US COMPANY"; finally, I write the output as csv.

In [ ]:
se_merge = pd.concat([se_us, se_non_us])

se_merge = se_merge.reset_index(drop = True)

se_merge.to_csv("/home/edoardo/SEmappingsDAFA_EdoardoPilla_7744044.csv", index = False)

I load the notebook on Drive to then convert it to html using nbconvert.

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

%shell jupyter nbconvert --to html /content/gdrive/MyDrive/projects/dafa/EdoardoPilla_7744044_case1.ipynb

#Drawbacks

Some US companies do not appear in the US dataframe, such as "Air Canada Inc", which is contained in CRSP.

This is because the filter based on business entity is uniquely relying on the contents of SECompanyName, and not on those of SEHeadline aswell, while the company only appears as "Air Canada" in SECompanyName, thus being wrongly excluded from the US dataframe.

Having tried to include SEHeadline as a filtering column, as shown by the code below, I ended up with a 300k entries dataframe, not shrank enough with respect to the original one, whose entries amount to 325k, to determine a sensible reduction in processing time, coupled with an increased risk of including non US companies in the US dataframe.

This is the reason why, eventually, I just relied on SECompanyName to filter the dataset, even though this came at the price of neglecting occurrences such as Air Canada Inc.

In [ ]:
se_us = se[(se["SECompanyName"].str.contains("|".join([" INC.", " INC", " Inc.", " Inc",
                                                   " LTD.", " LTD", " Ltd", " LP",
                                                   " PLC", " Plc", ".COM", ".Com"
                                                   " CO.", " CO", " Co.", " Co",
                                                   " CORP", " CORP.", " Corp", " Corp.",
                                                   " LLC", " Llc", " Lp", " Group",
                                                   " Trust", " plc"])))|
           (se["SEHeadline"].str.contains("|".join([" INC.", " INC", " Inc.", " Inc",
                                                       " LTD.", " LTD", " Ltd", " LP",
                                                       " PLC", " Plc", ".COM", ".Com"
                                                       " CO.", " CO", " Co.", " Co",
                                                       " CORP", " CORP.", " Corp", " Corp.",
                                                       " LLC", " Llc", " Lp", " Group",
                                                       " Trust", " plc"])))]
se_us = se_us.reset_index(drop = True)

Some non US companies still appear in the US dataframe, such as "International consolidated airlines SA", which is a spanish company, wrongly merged with an american one.

This is because the filter based on business entity type is too rough, since non american companies can also be labelled with typically american entities, such as LTD or CORP, but I still chose to proceed in this way, because originally I tried to extract any string ending within CRSP COMNAM column, assuming it would identify the company's entity, but ended up with 1131 unique endings, which would yield a US section with around 323k rows, thus not serving enough in terms of computation reduction.

This is also, intuitively, the reason why tests i carried out in the W/Y part of SE dataset are linked with lower accuracy, since many chinese companies are often labelled with CORP/CORPORATION, and start with W/Y.
<br><br><br>

Some US companies which appear in both dataframes are wrongly matched, such as for WIPRO and TARO; I believe that both instances are caused by the WRatio scorer, which weights according to the length of the substrings in order to return a match, and more specifically I think the first mismatch occurred because WIPRO is associated with "Limited" in SE, and is then matched with a company which has "Limited" in the related field in CRSP, even though it can be found within CRSP under "WIPRO LTD"; an ad hoc solution for this is to substitute "Limited" with "Ltd" within SE, something which I did for "Corporation" with "Corp" aswell, since it wasn't a rare occurrence, with the goal of making the entries in SE more similar to their CRSP counterparts.

Even though more than 1000 entries had "Incorporated" in their headline, I decided not to substitute it with "Inc", because I wanted to balance out the
potential mismatches caused by substituting "Limited" with "Ltd", since roughly 1000 entries had "Limited" in their headline.

Another similar situation occurred with "TARO PHARM CORP" as in CRSP, available in SE as "Taro Pharmaceuticals Industries", wrongly matched with "ACF INDUSTRIES" from CRSP; following the same logic, since "INDUSTRIES" is more valuable than "TARO" for the scorer, the mismatch occurred, but I didn't fix this as, contrary to the entity case described above, a correct match would have occurred by replacing "Industries" with "Corp", but this would have just fixed few mismatches and wouldn't have been applicable to the whole dataframe.

To this end, maybe something could have been done in order to filter the dataframes according to the relevant substrings' lengths, allowing WRatio to
work more effectively, but I couldn't come up with an idea to carry this out.
<br><br><br>

Before checking manually small test dataframes, I tried to add a column labelled "Similarity", where I computed the different fuzz ratios available
for the matches, but I realized that some matches which were visually correct, would have a low ratio (ex. partial ratio < 50 when using partial ratio
as scorer), whereas others would be visually wrong, but have a high ratio (ex. WRatio > 85 when using WRatio as scorer).

For this reason I checked manually to ascertain whether a match was correct or not, even though initially I tried to rely on the similarity ratios, because my idea was to label as mismatches all those entries whose related ratio (ex. WRatio when using WRatio as scorer) would be lower than a cutoff such as 70.
<br><br><br>

A potential flaw in my estimation is characterized by the fact that I set no condition to ensure that no overlapping intervals for the test dataframe
indexes would be picked (ex. potentially code could result in same test df to be generated multiple times, or two dataframes which include,
at some extent, the same entries such as df1 = se_us.iloc[0:1000] and df2 = se_us.iloc[600:1600]); no such occurrence happened for the test dataframes
I selected, but I still wanted to highlight this problem.

I believe the reason for this is that the original dataframe is around 275k rows, and having picked 11k, the likelihood of extracting overlapping
intervals was low enough.

#Questions

**How many companies have you managed to merge?**

Since extractOne left no missing values within the "MergeComnam" column, and given a 70% accuracy over the matching from my test dataframes, then the correctly merged headlines should amount to .7 * 275968 = 193178, where 275968 is the number of rows in se_us, the filtered dataset containing data about american companies according to my original assumption.

Assuming a better filtering, maybe more american headlines could have been included into se_us, and more non american headlines could have been excluded from it, yielding a better result.

Given the 48970 entries within CRSP, referring to 31803 unique PERMCOs, because each firm name must have at least one PERMCO, so the fact that the amount of entries is larger than the amount of PERMCOs means that some companies have had more than one name linked to them, then a proxy for the amount of merged companies could be .7 * 31803 = 22262, most likely overestimated due to the exclusion of some companies from se_us.
<br><br><br>

**What are the main obstacles to a perfect merge?**

The algorithm, based on Levenshtein distance, potentially can't reach a perfect merge (100% accuracy), because subtle differences in strings can lead to wrong matches, especially since it's not trivial to understand which scorer should be used to determine similarity between them.

In practice, these subtle differences can be determined by spelling mistakes, or by a different sorting of the tokens forming the string, even if both are semantically conveying the same message.

In the task's framework, a problem is given by the many different types of headline which appear in the SE dataset, essentially not always respecting a standardized structure, allowing the algorithm to work smoothly, or at least to preprocess the entries knowing that the changes will be effective for the whole dataset.
<br><br><br>

**What is special about CRSP names? Do you preprocess CRSP names
before merging them? If so, what changes do you make?**

CRSP names are in caps lock, and in some cases they may miss characters which are actually part of a company's name, as it is the case for 1-800 Contacts.com, available as 1 800 CONTACTS INC in crsp, namely without the "-" and "." signs.

However, I didn't really preprocess CRSP names, since the only query I used it for was to sort the business entities assuming that the last word was, indeed, the entity, but I aimed to make SEHeadline as similar as possible to CRSP, by removing any token which made the two strings different, for example the quarter and year information, and other special characters or potentially distracting words which may lead to mismatches; the approach I followed is explained in more detail in the Description section.
<br><br><br>

**Which tokens in the company names make your task challenging? How
did you deal with them?**

Having used the WRatio as scorer, short company names relative to the respective entity are often leading to mismatches as the scorer will value more the entity and thus match it to another company whose entity is written in the same way.

All the special characters, whitespace excluded, generally decreased the matching accuracy when using WRatio, while the accuracy increased when applying the partial ratio, although eventually what led me to obtain the best accuracy was to simply remove all of them, again excluding the whitespace, and run the algorithm using the default scorer.

Prior to removing quarter and year information, I also encountered some issues when dealing with companies having numbers in their names.
<br><br><br>

**What should be (theoretically) the number of records per firm’s fiscal
year? What is the actual number of observations per firm’s fiscal year?
Is this information helpful in identifying mismatches? Explain!**

To answer this question, I use a small and readable dataset containing earnings call information about the well known high tech company APPLE INC; the code below shows that the company was first inserted into CRSP in 1980 as "APPLE COMPUTER INC", to then be inserted again in 2007 as "APPLE INC".

If SE contained earnings calls information for the company since its inception in Q1 1981, up to 2020, the total amount of entries which should be visible in SE, directly related to the company, would then be (2020 - 1980) x 4 = 160, but this is not the case, since the first available data for apple's earnings call is Q2 2002, thus shrinking the total amount to (2020 - 2001) x 4 = 76 (since 2020 - 2002 would omit data for 1 year).

This is almost the case, in fact when filtering for the company name, SE yields 75 entries, in fact SE misses Q1 2002 information for the company; this implies that if after merging, APPLE COMPUTER INC and APPLE INC appear, jointly, more than 75 times into SE MergeComnam, one or more mismatches occurred.


In [ ]:
se_apple = se[se["SECompanyName"].str.contains("Apple Inc")]

crsp["DATE"] = pd.to_datetime(crsp["DATE"], format = "%Y%m%d")

print(crsp["COMNAM"][crsp["PERMCO"] == 7])

print(crsp["DATE"][crsp["PERMCO"] == 7])

More precisely, since data about Apple appears for the first time in SE in 2002, and data for one call is missing, given that the firm changed name at the beginning of 2007, 4 x 4 + 3 = 19 entries should be matched with APPLE COMPUTER INC, while 75 - 19 = 56 should be matched with APPLE INC.

This makes sense since from, and including, 2007 to 2020, 14 years passed, yielding 14 x 4 = 56 calls, which are all tracked into SE.

However, I haven't found a way to implement this reasoning for all the firms available, because SE isn't consistent in providing data for the same time span for each company, since some may have information starting in 2001, others starting in 2004 and so on, and some headlines may be missing from SE.
<br><br><br>

**What should be the relation between permco and the company name
from the SEHeadline? More specifically, can one SEHeadline’s company name be correctly linked to multiple permcos (i.e., to more than
one firm in CRSP)? Can one permco be correctly linked to multiple
company names from SEHeadline?**

Given that SEHeadline contains the company's name coherently in time (ex. APPLE COMPUTER INC before 2007, APPLE INC after 2007), while PERMCO is the unique company's identifier number, one SEheadline's name shouldn't be linkable to multiple PERMCOs, since multiple names represent the same company, but the opposite can be true (ex. APPLE COMPUTER INC. AND APPLE INC. both are linked to permco 7).

To better understand this, assume that 3 companies (A, B, C) are founded at t0 and hosted in CRSP; at t0, each company has one PERMCO (1, 2, 3), and one name; at t1 (1 year and thus 4 earnings calls), company A buys company B, but nothing will change within CRSP, whereas now information about earnings call from t1 onwards will only be available for companies A and C, so at t2 SEHeadline would contain 8 entries for company A and C, and 4 for company B.

If, at t2, company C changes its name to D, then a new name will be included into CRSP, and a new DATE will be uploaded too, notifying of the name change, so CRSP still hosts information about 3 companies, but C and D are now linked to PERMCO 3; at t3, 12 entries will characterize company A (which is A + B), 4 entries will characterize company B (which was bought by A and thus not updated anymore), and 12 entries will characterize company D (which was called C until t2, so 8 will be linked to C and 4 to D).
<br><br><br>

**What additional firm information would be helpful to check the validity
of a match?**

Knowing that CRSP only hosts data about US companies, knowing the firm's original nationality within SE could be helpful to determine if the names were matched correctly; in addition, as stated above, knowing when the first and last earnings call information was included into SE could aid into detecting mismatches by comparing the amount of mathes for each CRSP's company name, relative to CRSP's dates of name changes.

# Previous attempts

Some of the codes I've previously attempted are listed here as further reference.

In [ ]:
"""
# since the task states we should link the company name so that it's coherent at the time of the earnings call, it can be useful to convert the date column,
# from an integer into datetime, to see how company names evolved in time
crsp["DATE"] = pd.to_datetime(crsp["DATE"], format = "%Y%m%d")

# add a column to CRSP of shape "Qn_YYYY_COMNAM" to increase similarity between entries from CRSP and SE (Headline column)
q_year_comnam = []
for i in range(len(crsp["DATE"])):
    q_year_comnam.append("Q{} {} {}".format(crsp["DATE"][i].quarter, crsp["DATE"][i].year, crsp["COMNAM"][i]))

crsp["Q_YEAR_COMNAM"] = q_year_comnam

# add a column to SE based on SEHeadline, but without words including and after "Earnings", to potentially increase similarity between entries
se["SEHead"] = se['SEHeadline'].str.split('Earnings', 1).str[0].str.strip()

# will try to somehow merge for apple to check whether i understood the idea, and then repeat the process for all other names
# i already notice a possible issue since companies such as "Applegreen PLC" may be mistakenly involved in the fuzzy merge
# maybe somehow possible to group companies in crsp according to their permco as filter? even if firm changed name, permco stays the same
# also, permco can have multiple permno, but filtering directly on permco should avoid this problem?

print(crsp["COMNAM"][crsp["PERMCO"] == 7].unique())
print(crsp["DATE"][crsp["PERMCO"] == 7])

# apple appears in crsp dataset twice, once as apple computer inc in 1980, and once as apple inc in 2007;
# many other companies within crsp dataset contain "apple" in their names, but don't identify the famous high tech one (ex. some are called pineapple,
# some applebaum..); a way to get all the names tied with a specific company is by specifying the permco, which uniquely identifies them,
# regardless of their security type or other information; i could then try to pool all the possible company names by permco number,
# and somehow use this information to merge?

# create test datasets 
se_apple = se[(se["SEHeadline"].str.contains("Apple")) | (se["SEHeadline"].str.contains("apple"))]
se_apple = se_apple.reset_index(drop = True)

# having selected rows where "apple" appears aswell, i also include in the filtered dataset companies which have apple at some point in their names,
# and not just at the beginning

crsp_apple = crsp[crsp["COMNAM"].str.contains("APPLE")]
crsp_apple = crsp_apple.reset_index(drop = True)

# since the company names in crsp are always in uppercase letters, i selected "APPLE" instead of "Apple" or "apple";
# i will also try to rewrite the column to contain lowercase letters, and see if this affects the performance (maybe turning uppercase to lowercase
# is interpreted as an additional difference, so having to turn multiple letters into lowercase can affect performance sensibly?)

se_apple['MergeComnam'] = se_apple["SEHeadline"].apply(lambda x: process.extractOne(x, list(crsp_apple["COMNAM"]))[0])
se_apple

# this "naive" attempt allows to see several drawbacks of the approach, in a small and readable dataset;
# having specified nothing about a company's name changes, such as the year when this occurs, the applied name only depends on the string similarities;
# (ex. apple computer inc applied to apple fiscal year 2008, when it had already become apple inc); having seen above that apple transitioned from
# apple computer inc to apple inc on january 11, 2007, it's possible to see that the right thing occurs for all earnings calls in 2007, since the first
# occurred after the name changed, and the correct (new) name according to crsp is used; another issue occurs for the call named
# "AppleÂ® FY 10 Second Quarter Results Conference Call", where potentially the special characters included within trick the algorithm into thinking that
# this line is more similar to apple computer inc.

# moving on to other companies which contain "apple" in their names, a problem occurs for the earning call
# "Q3 2004 Applebee's&#174", part of DIne Brands Global inc, whose earning calls are termed "Applebee's international earnings call";
# since the characters &#174 are directly attached to "Applebee's", this also tricks the algorithm, and actually returns a blank entry, since it isn't
# able to connect it to any of the choices list (crsp names), even though the other earnings calls for the same company are correctly matched.

# further issues occur for the company "Applegreen PLC", whose earnings calls are coherently defined, but for which, probably, no counterpart in crsp is
# included, and this leads the algorithm to recognize it as "APPLETREE INC", which is a different company.

# another company where this happens is "Oldapco Inc", whose earnings call keyword is "Appleton"; this gets matched with apple inc, or with apple computer
# inc when the earnings calls are termed "Appleton Partners" instead of just "Appleton"; it may be the case that this company also isn't available in crsp?

# the last problem in this test dataset occurs with the company "Keurig Dr Pepper Inc", whose earnings calls are termed "Dr Pepper Snapple Group Inc";
# these are at first correctly matched with the corresponding crsp name, but some entries have a comma (,) after the word Group, and this leads the
# algorithm to match them with Apple Computer Inc, interesting since the comma is the only difference, but leads to moving from the correct name to the
# wrong one

crsp[crsp["COMNAM"].str.contains("APPLEGREEN")]

# indeed, crsp dataset seems not to contain any applegreen company, potential reason why it gets matched with appletree instead

crsp[crsp["COMNAM"].str.contains("OLDAPCO")]

# same seems to happend for oldapco, as written above; not much can be done for data which simply isn't available in crsp, except maybe establishing
# a lower bound for the matching accuracy, over which the match shouldn't occur at all? (ex. don't match for scorer_cutoff < 40)

# will try to see what happens if i remove special characters such as & or #, which led to a blank entry before
se_apple["SEHeadline"] = se_apple["SEHeadline"].str.replace("&#174", "")
se_apple['MergeComnam'] = se_apple["SEHeadline"].apply(lambda x: process.extractOne(x, list(crsp_apple["COMNAM"]))[0])
se_apple

# this ad hoc replacement wasn't enough to prevent the blank entry, i should probably replace it with the proper headline name,
# but this would configure a hard coding solution, which is not suitable here since probably many such situations occur in the full dataset

# removing comma from headline
se_apple["SEHeadline"] = se_apple["SEHeadline"].str.replace(",", "")
se_apple['MergeComnam'] = se_apple["SEHeadline"].apply(lambda x: process.extractOne(x, list(crsp_apple["COMNAM"]))[0])
se_apple

# solved the problem encountered in last company without having to change scorer in extractone

# removing all non alphanumeric characters, except white space
se_apple["SEHeadline"] = se_apple["SEHeadline"].str.replace(r"[^a-zA-Z\d\s:]", "")
se_apple['MergeComnam'] = se_apple["SEHeadline"].apply(lambda x: process.extractOne(x, list(crsp_apple["COMNAM"]))[0])
se_apple

# serves same function as removing comma, but also solves the "&#174" problem, since apparently replacing all non alphanumeric characters except space,
# allowed to visualize the rest of the text in that entry, thus allowing to match correctly for applebee; still didn't solve Apple fy 10 second quarter
# line, since it only removed the special characters "Â®", but this wasn't enough to allow the correct match

# adding on the last problem which wasn't solved, from the test dataset it's possible to see how the headline
# "Apple FY 06 Fourth Quarter Results Conference Call" is matched with apple computer inc, which apparently is correct since in 2006 the company's name
# was still that, but then the same headline, where only the year changes (ex. Apple FY 08 First Quarter Results Conference Call), still matches
# with the old name, implying that probably the algorithm is tricked by the fact that Q4 or Q1 are written extensively;
# a solution to this may be to drop FY and replace First Quarter with Q1, and so on for other quarters
se_apple["SEHeadline"] = se_apple["SEHeadline"].str.replace("First Quarter", "Q1")
se_apple["SEHeadline"] = se_apple["SEHeadline"].str.replace("FY", "")
se_apple['MergeComnam'] = se_apple["SEHeadline"].apply(lambda x: process.extractOne(x, list(crsp_apple["COMNAM"]))[0])
se_apple

# doesn't really make a difference; i also tried to use fuzz.partial_ratio as a scorer, which basically inverted the problem, namely matching apple
# instead of apple computer, regardless of date, but overall performance also decreased

# changing crsp comnam to lowercase to see how performance is affected
crsp_apple["COMNAM"] = crsp_apple["COMNAM"].str.lower()
se_apple['MergeComnam'] = se_apple["SEHeadline"].apply(lambda x: process.extractOne(x, list(crsp_apple["COMNAM"]))[0])
se_apple

# performance apparently doesn't change, at least for the test dataset, as matches are exactly the same as when keeping comnam in uppercase

# will try to see how things change when adding a scorer to extractone, knowing the default one is WRatio
se_apple['MergeComnam'] = se_apple["SEHeadline"].apply(lambda x: process.extractOne(x, list(crsp_apple["COMNAM"]), scorer = fuzz.token_set_ratio)[0])
se_apple

# processing time decreased from 8/9 seconds to 1 second, but performance decreased sensibly, more mismatches

# creating test datasets for special character case
se_cont = se[(se["SEHeadline"].str.contains("Contacts")) | (se["SEHeadline"].str.contains("CONTACTS"))]
se_cont = se_cont.reset_index(drop = True)

crsp_cont = crsp[crsp["COMNAM"].str.contains("CONTACTS")]
crsp_cont = crsp_cont.reset_index(drop = True)

# makes me think that removing all special characters except for whitespace is a rough filter, should also exclude dots and - to avoid having issues with
# such companies such as 1-800 contacts

se_cont_special = se_cont[se_cont["SEHeadline"].str.contains(r"[^a-zA-Z\d\s:]")]
se_cont_special = se_cont_special.reset_index(drop = True)
se_cont_special['MergeComnam'] = se_cont_special["SEHeadline"].apply(lambda x: process.extractOne(x, list(crsp_cont["COMNAM"]),
                                 scorer = fuzz.partial_ratio)[0])

# splitting df to isolate entries containing special characters allows to correctly match using partial ratio,
# even without replacing special characters with blank spaces (at least in this case, but method may generally be too rough to be applied to whole df)

se_cont_special = se_cont[se_cont["SEHeadline"].str.contains(r"[^a-zA-Z\d\s:]")]
se_cont_special = se_cont_special.reset_index(drop = True)
se_cont_special["SEHeadline"] = se_cont_special["SEHeadline"].str.replace(r"[^a-zA-Z\d\s:]", "")
se_cont_special['MergeComnam'] = se_cont_special["SEHeadline"].apply(lambda x: process.extractOne(x, list(crsp_cont["COMNAM"]),
                                 scorer = fuzz.partial_ratio)[0])

# this cell shows that the match is still correct, even after dropping the special characters; this may be helpful to fix those entries
# having &#174 in the headline, which prevent from identifying the correct company


se_cont["SEHeadline"] = se_cont["SEHeadline"].str.replace(r"[^a-zA-Z\d\s:]", "")
se_cont['MergeComnam'] = se_cont["SEHeadline"].apply(lambda x: process.extractOne(x, list(crsp_cont["COMNAM"]), scorer = fuzz.partial_ratio)[0])
else:
  se_cont['MergeComnam'] = se_cont["SEHeadline"].apply(lambda x: process.extractOne(x, list(crsp_cont["COMNAM"]))[0])

# actually, the problem here is that the crsp dataset includes 1-800 contacts as 1 800 contacts, namely without -; this leads to the wrong match,
# regardless of the filtering applied above which removes -; actually, if the special characters were to be replaced by an empty space,
# would this change something?

# UPDATE: replacing with an empty space didn't change anything, but using the partial_ratio as a scorer fixed this problem;
# this can be an hint for situations where companies have numbers in their names?

# split SE into two parts: one whose headline column contains special characters (excluding whitespace), one not containing them
se_special = se[se["SEHeadline"].str.contains(r"[^a-zA-Z\d\s:]")]
se_no_special = se[~se["SEHeadline"].str.contains(r"[^a-zA-Z\d\s:]")]
"""